In [ ]:
import psycopg2
import import_ipynb
from Utils import *
import os
from dotenv import load_dotenv

# Cargar variables del archivo .env
load_dotenv()

connection_params = {
    'dbname': os.getenv("DB_NAME"),
    'user': os.getenv("DB_USER"),
    'password': os.getenv("DB_PASSWORD"),
    'host': os.getenv("DB_HOST"),
    'port': os.getenv("DB_PORT")
}

print("Parámetros de conexión cargados.")

In [ ]:
# Conectar a la base de datos
try:
    conn = psycopg2.connect(**connection_params)
    cur = conn.cursor()
    print("Conexión a la base de datos exitosa.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")
    raise

In [ ]:
# Transferir datos únicos a paciente_curated (con actualización si existe)
try:
    cur.execute("""
    INSERT INTO paciente_curated (nombre1, nombre2, nombre3, apellido1, apellido2, tipo_de_identificacion, identificacion, sexo, fecha_nacimiento, edad, municipio_residencia, codigo_residencia, barrio_residencia, direccion, id_paciente, etiqueta, fecha_ingreso_sistema)
    SELECT DISTINCT nombre1, nombre2, nombre3, apellido1, apellido2, tipo_de_identificacion, identificacion, sexo, fecha_nacimiento, edad, municipio_residencia, codigo_residencia, barrio_residencia, direccion, id_paciente, etiqueta, fecha_ingreso_sistema
    FROM paciente
    ON CONFLICT (id_paciente) DO UPDATE SET
        nombre1 = EXCLUDED.nombre1,
        nombre2 = EXCLUDED.nombre2,
        nombre3 = EXCLUDED.nombre3,
        apellido1 = EXCLUDED.apellido1,
        apellido2 = EXCLUDED.apellido2,
        tipo_de_identificacion = EXCLUDED.tipo_de_identificacion,
        identificacion = EXCLUDED.identificacion,
        sexo = EXCLUDED.sexo,
        fecha_nacimiento = EXCLUDED.fecha_nacimiento,
        edad = EXCLUDED.edad,
        municipio_residencia = EXCLUDED.municipio_residencia,
        codigo_residencia = EXCLUDED.codigo_residencia,
        barrio_residencia = EXCLUDED.barrio_residencia,
        direccion = EXCLUDED.direccion,
        etiqueta = EXCLUDED.etiqueta,
        fecha_ingreso_sistema = EXCLUDED.fecha_ingreso_sistema;
    """)
    pacientes_insertados = cur.rowcount
    print(f"Pacientes insertados/actualizados en paciente_curated: {pacientes_insertados}")
except Exception as e:
    print(f"Error al transferir pacientes: {e}")
    conn.rollback()
    raise

In [ ]:
# Transferir datos únicos a infolab_curated, relacionando con paciente_curated
try:
    cur.execute("""
    INSERT INTO infolab_curated (fecha_toma_muestra, fecha_ingreso, fecha_informe, entidad, eps, servicio, muestra_remitida, descripcion_macroscopica, descripcion_microscopica, diagnostico, comentario, id_paciente, archivo, fuente, cie10, edad, sexo, diagnostico_clinico, observacion, medico_ordenante, medico_remitente)
    SELECT DISTINCT i.fecha_toma_muestra, i.fecha_ingreso, i.fecha_informe, i.entidad, i.eps, i.servicio, i.muestra_remitida, i.descripcion_macroscopica, i.descripcion_microscopica, i.diagnostico, i.comentario, i.id_paciente, i.archivo, i.fuente, i.cie10, i.edad, i.sexo, i.diagnostico_clinico, i.observacion, i.medico_ordenante, i.medico_remitente
    FROM infolab i
    INNER JOIN paciente_curated p ON i.id_paciente = p.id_paciente
    WHERE NOT EXISTS (
        SELECT 1 FROM infolab_curated ic
        WHERE ic.id_paciente = i.id_paciente
          AND ic.servicio = i.servicio
          AND ic.muestra_remitida = i.muestra_remitida
          AND ic.descripcion_macroscopica = i.descripcion_macroscopica
          AND ic.descripcion_microscopica = i.descripcion_microscopica
          AND ic.observacion = i.observacion
    );
    """)
    infolab_insertados = cur.rowcount
    print(f"Registros insertados en infolab_curated: {infolab_insertados}")
except Exception as e:
    print(f"Error al transferir infolab: {e}")
    conn.rollback()
    raise

In [ ]:
# Confirmar cambios y cerrar conexión
try:
    conn.commit()
    print("Transferencia completada exitosamente.")
except Exception as e:
    print(f"Error al confirmar cambios: {e}")
    conn.rollback()
    raise
finally:
    cur.close()
    conn.close()
    print("Conexión cerrada.")